Author: "Shivani Khandelwal"

Reference: Murtaza Hassan - https://youtu.be/0IqCOPlGBTs

In [1]:
import cv2
import numpy as np

In [2]:
def stackImages(img_array, scale):
    rows = len(img_array)
    cols = len(img_array[0])

    rowsAvailable = isinstance(img_array[0], list)
    height = img_array[0][0].shape[0]
    width = img_array[0][0].shape[1]

    if rowsAvailable:
        for r in range(0, rows):
            for c in range(0, cols):
                img_array[r][c] = cv2.resize(img_array[r][c], (0, 0), None, scale, scale)
                if len(img_array[r][c].shape) == 2:
                    img_array[r][c] = cv2.cvtColor(img_array[r][c], cv2.COLOR_GRAY2BGR)

        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(img_array[x])
        ver = np.vstack(hor)
        
    else:
        for x in range(0, rows):
            img_array[x] = cv2.resize(img_array[x], (0, 0), None, scale, scale)
            if len(img_array[x].shape) == 2:
                img_array[x] = cv2.cvtColor(img_array[x], cv2.COLOR_GRAY2BGR)

        hor = np.hstack(img_array)
        ver = hor

    return ver


def rectContour(contours):
    rectCon = []
    for con in contours:
        area = cv2.contourArea(con)
        if area > 50:
            peri = cv2.arcLength(con, True)
            approx = cv2.approxPolyDP(con, 0.02*peri, True)
            if len(approx) == 4:
                rectCon.append(approx)
                
    rectCon = sorted(rectCon, key=cv2.contourArea, reverse=True)
    return rectCon


def reorderPoints(points):
    points = points.reshape((4, 2))
    ordered_points = np.zeros((4, 1, 2), np.int32)

    add = np.sum(points, axis=1)
    ordered_points[0] = points[np.argmin(add)]
    ordered_points[3] = points[np.argmax(add)]
    
    diff = np.diff(points, axis=1)
    ordered_points[1] = points[np.argmin(diff)]
    ordered_points[2] = points[np.argmax(diff)]

    return ordered_points


def splitBoxes(img):
    rows = np.vsplit(img, 5)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 5)
        boxes.extend(cols)
        
    return boxes

In [3]:
omr1 = cv2.imread('images/omr_1.jpg')
print(omr1.shape)

height = 700
width = 700

questions = 5
choices = 5
answers = [1, 2, 0, 3, 4]

(1600, 1200, 3)


In [4]:
omr1 = cv2.resize(omr1, (width, height))

omr1_gray = cv2.cvtColor(omr1, cv2.COLOR_BGR2GRAY)
omr1_blur = cv2.GaussianBlur(omr1_gray, (5, 5), 1) # Sigma (Std.) defines the amount of blurring - 1
omr1_canny = cv2.Canny(omr1_blur, 10, 50)
print(omr1_canny.shape)

(700, 700)


In [5]:
contours, hierarchy = cv2.findContours(omr1_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
omr1_contours = cv2.drawContours(omr1.copy(), contours, -1, (0, 255, 0), 3)

In [14]:
rect_con = rectContour(contours)

marking_area = rect_con[0]
grading_area = rect_con[1]

if marking_area.size != 0 and grading_area.size != 0:
    omr1_selected_area = cv2.drawContours(omr1.copy(), marking_area, -1, (255, 0, 0), 10)
    omr1_selected_area = cv2.drawContours(omr1_selected_area, grading_area, -1, (0, 0, 255), 10)

    marking_area = reorderPoints(marking_area)
    grading_area = reorderPoints(grading_area)

    marking_pt1 = np.float32(marking_area)
    marking_pt2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv2.getPerspectiveTransform(marking_pt1, marking_pt2)
    marking_res = cv2.warpPerspective(omr1.copy(), matrix, (width, height))

    grading_pt1 = np.float32(grading_area)
    grading_pt2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv2.getPerspectiveTransform(grading_pt1, grading_pt2)
    grading_res = cv2.warpPerspective(omr1.copy(), matrix, (width, height))

    marking_res_gray = cv2.cvtColor(marking_res, cv2.COLOR_BGR2GRAY)
    marking_res_thres = cv2.threshold(marking_res_gray, 170, 255, cv2.THRESH_BINARY_INV)[1]

    boxes = splitBoxes(marking_res_thres)
    pixel_val = np.zeros((questions, choices))

    for b in range(len(boxes)):
        total_pixels = cv2.countNonZero(boxes[b])
        if total_pixels > 5000:
            row = int(b/questions)
            col = b % choices
            pixel_val[row][col] = 1

    grading = []
    for i in range(questions):
        index = np.where(pixel_val[i] == 1)[0]
        if len(index) == 0:
            grading.append(-1)
        else:
            grading.append(index[0])

    score = 0
    for i in range(questions):
        if grading[i] == answers[i]:
            score += 1

    final_score = (score/questions)*100

[1, 2, 0, 0, 4]
4


In [10]:
img_blank = np.zeros_like(omr1)
img_array = [[omr1, omr1_gray, omr1_blur, omr1_canny],
             [omr1_contours, omr1_selected_area, marking_res, marking_res_thres]]
img_stack = stackImages(img_array, 0.5)

cv2.imshow('Stacked Images', img_stack)
cv2.waitKey(0)
cv2.destroyAllWindows()